In [1]:
from GameEnvironment import ModelGameEnvironment
import torch
from ActorCritic import ActorCritic, Actor, Critic

In [2]:
game = ModelGameEnvironment(.5, 2, .99, 10)

In [3]:
game.reset()


(tensor([50., 50.]), None, False)

In [4]:
print(game.step(torch.Tensor([10,10]) ))
print(f"quantity history {game.quantity}")
print(f"profit history {game.profit}")

(tensor([10., 10.]), tensor([-195., -195.]), False)
quantity history tensor([[10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
profit history tensor([[-195.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
        [-195.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]])


In [5]:
env = ModelGameEnvironment(.5, 2, .99, 10)

# Initialize actor and critic
input_dim = 2

actor = Actor(input_dim)
critic = Critic(input_dim)

# Initialize actor-critic algorithm
agent = ActorCritic(actor, critic, .99)

# Training loop
num_episodes = 1000
for episode in range(num_episodes):
    state, _, done = env.reset()

    
    while not done:
        action_profile = torch.Tensor(2)
        log_probs = []


        for _ in range(2):
            action, log_prob = agent.select_action(state)
            action_profile[_] = action
            log_probs.append(log_prob)
        
        
        # Compute reward based on the action profile
        next_state, reward_tensor, done = env.step(action_profile)
        reward = torch.sum(reward_tensor).item()
        


        for i in range(2):
            agent.update(state, log_probs[i], reward, next_state)
        
        # End episode if a stopping criterion is met (optional)
        # For normal form games, typically one step per episode is enough
        done = True

    if episode % 100 == 0:
        print(f"Episode {episode} complete")

/Users/pacoaristi/Desktop/Github ferdos/EquiLearn_Sberg/classes/ActorCritic.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reward = torch.tensor(reward, dtype=torch.float32, device=self.device)


tensor([5.6543])


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn